Point to the correct directory

In [1]:
import sys
import os
from dotenv import load_dotenv

# Get the absolute path to the project root (the parent of notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add it to sys.path so Python can find 'src'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import
from src.services.ai_processor import AIProcessor
from models.pydantic_classes import *


In [2]:
from src.config import BASE_URL

load_dotenv()

True

Example usage: Normal response

In [4]:
processor = AIProcessor(
    base_url=BASE_URL, 
    api_key=os.getenv("API_KEY"), 
    client=os.getenv("CLIENT_ID"))

In [4]:
# Can specigy model, defaults to gpt-4
response = processor.generate_completion(
    system_prompt="You will receive a list of cities and countries in the user message. Your job is to extract valid capital-city and country pairs",
    user_prompt="paris rome valencia madrid germany berlin tolouse madrid italy france",
    model="gpt-4o",
    temperature=0,
    max_tokens=1000
)

print(response)

content='Here are the valid capital-city and country pairs from the list:\n\n1. Paris, France\n2. Rome, Italy\n3. Berlin, Germany\n4. Madrid, Spain' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 50, 'total_tokens': 87, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-dev-default', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-C0pemDxtNeKgnBQXaDIHMUuIzPu3V', 'service_tier': None, 'finish_reason': 'Stop', 'logprobs': None, 'content_filter_results': {}} id='run--4a485e1e-d189-44b0-b721-d1b316966cff-0' usage_metadata={'input_tokens': 50, 'output_tokens': 37, 'total_tokens': 87, 'input_token_details': {}, 'output_token_details': {}}


Example usage: Structured output

In [5]:
response = processor.generate_completion(
    system_prompt="You will receive a list of cities and countries in the user message. Your job is to extract valid capital-city and country pairs",
    user_prompt="paris rome valencia madrid germany berlin tolouse madrid italy france",
    pydantic_schema=Response,
    temperature=0,
    max_tokens=1000
)

In [6]:
print(response)

{'response': [{'city': 'Paris', 'country': 'France'}, {'city': 'Rome', 'country': 'Italy'}, {'city': 'Berlin', 'country': 'Germany'}]}


Example usage: Getting the embeddings

In [ ]:
response = processor.generate_embeddings(
    input_text="paris rome valencia madrid germany berlin tolouse madrid italy france"
)

Example usage: Agents

In [8]:
# List available agents
agents = processor.list_agents()
print(agents)

[]


In [9]:
# Create an agent
processor.create_agent(
    agent_name = "geolocation agent",
    description = "Geo agent for geocoding and computing routes between locations.",
    instructions = (
        "You are a tool-using assistant designed specifically for geocoding (finding coordinates for an address or vice versa) and computing routes between locations.\n\n"
        "Strictly follow these rules:\n"
        "- Your responses must be very concise and strictly professional—they are intended for GIS/business users, not for consumer or traveler enquiries.\n"
        "- You must ALWAYS call the correct tool for every actionable user request, never answer directly from your own knowledge.\n"
        "- Use the geocoding tool for address-to-coordinates or coordinates-to-address queries. If the user provides an address or a pair of latitude and longitude, call the geocoding tool immediately and reply with its result.\n"
        "- Use the route computation tool for routing/distance/duration queries between two or more locations. If the user specifies a travel mode (Drive, Walk, Bicycle, Transit), use it. If not specified, use Drive as the default, mention this fact, and perform the tool call and reply with its result.\n"
        "- DO NOT ask users to confirm if they clearly provided sufficient info for a tool call—proceed to call the tool and answer.\n"
        "- If required input is genuinely ambiguous or missing (e.g., no address or coordinates or not enough data for the tool call), reply with 'Unkown'. Do NOT reply with a general invitation to provide info or with process confirmations like 'Let me check that...' or 'I'll process your request now.'\n"
        "- Do NOT make up data—use only user input and information or results provided by tools within the conversation history.\n"
        "- Honor specific user preferences (e.g., request for pedestrian route) if mentioned.\n"
        "- NEVER reply with meta messages, generic confirmations, or intent restatements. Respond only with a tool result OR a specific data request.\n"
        "\nExamples:\n"
        "- User: 'What address is at latitude 51.5237 and longitude -0.1585?'\n"
        "  Agent: (call geocoding tool using those coordinates, respond concisely with the result)\n"
        "- User: 'Directions from A to B.'\n"
        "  Agent: (call route computation tool with default mode Drive, respond concisely with the result, including a note about defaulting to Drive)\n"
    ),
    agent_kind = "Geo",
    prompt_template = (
        "You are a location intelligence assistant. Always provide concise, professional, tool-based answers for GIS/work users. Never answer from your own knowledge or with conversational padding."
    )
)

{'agentName': 'geolocation agent',
 'clientId': 'middlemarketgb',
 'instructions': "You are a tool-using assistant designed specifically for geocoding (finding coordinates for an address or vice versa) and computing routes between locations.\n\nStrictly follow these rules:\n- Your responses must be very concise and strictly professional—they are intended for GIS/business users, not for consumer or traveler enquiries.\n- You must ALWAYS call the correct tool for every actionable user request, never answer directly from your own knowledge.\n- Use the geocoding tool for address-to-coordinates or coordinates-to-address queries. If the user provides an address or a pair of latitude and longitude, call the geocoding tool immediately and reply with its result.\n- Use the route computation tool for routing/distance/duration queries between two or more locations. If the user specifies a travel mode (Drive, Walk, Bicycle, Transit), use it. If not specified, use Drive as the default, mention this

In [10]:
agent_response = processor.execute_agent(
    handler_name = "geolocation agent",
    user_message = "What are the coordinates of the tower of london",
    agent_kind = "Geo")

print(agent_response)

{'response': 'Coordinates of the Tower of London: Latitude 51.5081, Longitude -0.0759.', 'conversation': []}


In [11]:
agent_response = processor.execute_agent(
    handler_name = "geolocation agent",
    user_message = "What is the address for 40.748817, -73.985428?",
    agent_kind = "Geo")

print(agent_response)

{'response': 'The address for latitude 40.748817 and longitude -73.985428 is W 34 St & 5 Av, New York, NY 10001, USA.', 'conversation': []}


Example usage: Agent groups

In [12]:
# Create a second agent for the group
processor.create_agent(
    agent_name = "tour agent",
    description = "General agent for recomending activities in a given location.",
    instructions = (
        "You are an expert tour guide. You will be provided with an address\n\n"
        "Your task will be to provide touristy activities to do nearby the provided address.\n\n"
        "Avoid including restaurants or shops, and prioritize touristy locations"
    ),
    agent_kind = "General",
    prompt_template = (
        "You are a touring assistant. Always provide concise, professional, tool-based answers for GIS/work users."
    )
)

{'agentName': 'tour agent',
 'clientId': 'middlemarketgb',
 'instructions': 'You are an expert tour guide. You will be provided with an address\n\nYour task will be to provide touristy activities to do nearby the provided address.\n\nAvoid including restaurants or shops, and prioritize touristy locationsThe user question to process:\n---\n{{payload.userMessage}}\n',
 'promptTemplate': 'You are a touring assistant. Always provide concise, professional, tool-based answers for GIS/work users.',
 'agentKind': 'General',
 'promptId': '4c2452e3-542a-4525-b1cb-5aab78a7e421',
 'promptKind': 'Liquid',
 'maxTokens': 1000,
 'temperature': 0.3,
 'description': 'General agent for recomending activities in a given location.',
 'modelDeploymentName': 'gpt-4o-dev-default'}

In [15]:
agent_group = processor.create_agent_group(
    agent_group_name="geo touring assistant",
    
    orchestrator_instruction=(
        "You are the orchestrator named 'Geo Touring Assistant'. The user will send a set of coordinates. Your job will be to recomend touristy activities near the location provided by the user.\r\n"
        "Strictly follow these instructions step-by-step:\r\n"
        "1) Use the **geolocation agent** to obtain the address and city belonging to the set of coordinates provided by the user"
        "2) If the geolocation agent does not return an address - ask him directly to provide an answer.\r\n"
        "3) Then, send the response of the geolocation agent to the **tour agent** to obtain the touristy activities nearby that area"
        "4) If the user query is not a set of coordinates, respond: 'please provide a set of coordinates'\r\n"
    ),
    
    selection_instruction=(
        "Choose only from these participants:\r\n"
        "- geo touring assistant\r\n"
        "- geolocation agent\r\n"
        "- tour agent\r\n\r\n"
        "Always follow these recomendations when selecting the next participant:\r\n"
        "1) After user input, first call the **geolocation agent**.\r\n"
        "2) Once the geolocation agent returns a valid address, call the **tour agent** using the address returned by the geolocation agent in the prompt.\r\n"
        "3) Once the **tour agent** responds, evaluate the quality of the conversation.\r\n"
        "4) If the answer is approved, the conversation ends.\r\n"
        "5) If the answer is rejected, call the geolocation agent again."
    ),
    
    result_quality_control_instruction=(
        "You can only respond with one of the following: 'approved' or 'rejected'.\r\n"
        "Never use your own knowledge to answer the user's question. Never make up information.\r\n"
        "Rules for evaluating:\r\n"
        "1) If the tour agent gives a logical answer and does not ask for more help, respond with 'approved'.\r\n"
        "2) If the tour agent gives an illogical answer or asks for additional help, respond with 'rejected'.\r\n"
        "3) If there is no response from the tour agent, respond with 'rejected'."
    ),
    
    agents=[
        {"agentName": "geolocation agent", "agentKind": "Geo"},
        {"agentName": "tour agent", "agentKind": "General"}
    ]
)


In [18]:
agent_response = processor.execute_agent(
    agent_name = "geo touring assistant",
    user_message = "40.748817, -73.985428?",
    agent_kind="Group")

print(agent_response)

{'response': "This location is near the Empire State Building in New York City. Here are some touristy activities you can enjoy nearby:\n\n1. **Empire State Building**: Visit the iconic skyscraper and enjoy breathtaking views of New York City from its observation decks.\n\n2. **Bryant Park**: A beautiful urban park offering a relaxing atmosphere with seasonal activities such as ice skating in the winter and outdoor movies in the summer.\n\n3. **New York Public Library**: Explore the stunning architecture and vast collection of books at this historic library.\n\n4. **Madison Square Garden**: Check out the schedule for concerts, sports events, and other entertainment happening at this famous venue.\n\n5. **Macy's Herald Square**: While primarily a shopping destination, Macy's also hosts events and has a rich history worth exploring.\n\n6. **Times Square**: Experience the vibrant lights and bustling atmosphere of one of the most famous intersections in the world.\n\n7. **The Museum of Mod

In [14]:
processor.delete_agent_group("geo touring assistant")


b''